In [24]:
import pandas as pd
import numpy.random as npr
import matplotlib.pyplot as plt
import ssm
import utils as uls
import numpy as np
import seaborn as sns
from sklearn.metrics import accuracy_score

import sys
sys.path.append('..')
import meals as ml

npr.seed(0)

In [25]:
def in_meal(meals: list, time) -> bool:
    for each in meals:
        if time >= each[0] and time <= each[1]:
            return True
    return False

In [30]:
data = pd.read_csv(
    '../../behavior data integrated/Bhv 7 - Ctrl/M2/Contingency Flip/FED000_071823_00.CSV')

data = data[['MM:DD:YYYY hh:mm:ss', 'Event', 'Active_Poke', 'Pellet_Count']].replace(
    {'LeftWithPellet': 'Left', 'LeftDuringDispense': 'Left',
     'RightWithPellet': 'Right', 'RightDuringDispense': 'Right'}).rename(
    {'MM:DD:YYYY hh:mm:ss': 'Time'}, axis='columns')
data['Time'] = pd.to_datetime(data['Time'])

meals = ml.find_meals(data)

data = data[data['Event'] != 'Pellet']

data['prev_event'] = data['Event'].shift(fill_value=None)
data['prev_active'] = data['Active_Poke'].shift(fill_value=None)
data['meal'] = [in_meal(meals, each) for each in data['Time']]
print(data['meal'])

# Extract the date and time
data['Date'] = data['Time'].dt.date
data['Time_of_day'] = data['Time'].dt.time

# second day
unique_dates = data['Date'].unique()
second_day = unique_dates[1]
third_day = unique_dates[2]

# Filter rows for the second day between 7:00 a.m. and 11:00 a.m.
start_time = pd.to_datetime('07:00:00').time()
end_time = pd.to_datetime('13:00:00').time()

second_day_data = data[(data['Date'] == second_day) & (
    data['Time_of_day'] >= start_time) & (data['Time_of_day'] <= end_time)]

third_day_data = data[(data['Date'] == third_day) & (
    data['Time_of_day'] >= start_time) & (data['Time_of_day'] <= end_time)]

selected_rows = pd.concat([second_day_data, third_day_data], axis='index')
selected_rows = selected_rows.drop(['Date', 'Time_of_day', 'Time'], axis=1)

X = selected_rows.drop(['Event'], axis='columns')
y = selected_rows['Event']

mapper = {'Left': 0, 'Right': 1}

X['Active_Poke'] = X['Active_Poke'].map(mapper)

X['prev_event'] = X['prev_event'].map(mapper)
X['prev_active'] = X['prev_active'].map(mapper)

X['bias'] = 1
y = y.map(mapper)
Y = []
for each in y:
    Y.append([each])
# return X.to_numpy(), np.array(Y)
X.to_numpy()


0       False
1       False
3       False
4        True
6        True
        ...  
1573     True
1575    False
1577    False
1579    False
1581    False
Name: meal, Length: 1066, dtype: bool


array([[1, 162, 0, 1, False, 1],
       [1, 162, 0, 1, False, 1],
       [1, 162, 0, 1, False, 1],
       ...,
       [0, 386, 0, 0, False, 1],
       [0, 387, 0, 0, False, 1],
       [0, 387, 1, 0, False, 1]], dtype=object)